<a href="https://colab.research.google.com/github/jyj0607040607/AIStudy24/blob/master/%EA%B5%90%EC%B0%A8%EA%B2%80%EC%A6%9D_%EA%B7%B8%EB%A6%AC%EB%93%9C%EC%84%9C%EC%B9%98_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

지금까지 배운 알고리즘은 훈련세트와 테스트를 나누어 평가를 진행하였다. 하지만, 결국 테스트로 평가를 하면 테스트세트에 맞는 모델이 만들어 지는 결론이 나온다.

테스트 세트에 일반화 성능을 올바르게 예측하려면 가능한 한 테스트세트를 사용하지 말아야한다.

모델을 만들고 나서 마지막에 딱 한번 사용하는 것이 좋다.

결정트리에서 max_depth 매개변수를 사용한 하이퍼파라미터 튜닝해보자.



In [ ]:
# 검증세트
# 테스트 세트를 사용하지 않으려면 모델이 과대적합인지 과소적합인지 판단하기 어렵다.
# 테스트 세트를 사용하지 않고, 측정하는 간단한 방법은 훈련세트를 또 나누는 것이다.
# 훈련세트 60%, 검증세트 20%, 테스트세트 20% -> 실무에서 많이 쓰는 기법

# 테스트하고 싶은 매개변수를 바꿔가면서 가장 좋은 모델이 나올 수있다.
# 이 매개변수를 사용해 훈련세트와 검증세트를 합쳐 전체 훈련 데이터에서 모델을 다시 훈련한다.
# 마지막 테스트에서 최종 점수를 평가한다.
# 훈련세트에서 모델을 훈련하고, 검증세트로 모델을 평가한다.
# 마지막 세트에서 테스트세트를 이용해 최종 점수를 평가한다.

In [ ]:
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

from sklearn.model_selection import train_test_split
# 훈련세트와 테스트 세트를 나눔
train_input, test_input, train_target, test_target = train_test_split(
    data, target, test_size=0.2, random_state=42) # 20%

#검증세트 20% 생성
sub_input, val_input, sub_target, val_target = train_test_split(
    train_input, train_target, test_size=0.2, random_state=42)

print(sub_input.shape, val_input.shape)
# 원래 5197개였던 훈련세트가 4157개로 줄고, 검증세트가 1040개가 됨

(4157, 3) (1040, 3)


In [ ]:
# sub_input, sub_target과 val_target을 사용해 모델을 만들고 평가한다.
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)  # 훈련

print(dt.score(sub_input, sub_target))    # 출력 0.9971133028626413(과대적합)
print(dt.score(val_input, val_target))    # 검증 86.44%
# 매개변수를 바꿔서 더 좋은 모델을 찾아야한다.

0.9971133028626413
0.864423076923077


In [ ]:
# 검증세트를 만드느냐고 훈련 세트가 줄어들었다. (많은 데이터를 훈련하면 좋은 결과가 나온다.)
# 교차 검증 : 검증 세트를 떼어 내어 평가하는 과정을 여러번 반복한다.
# [훈련세트, 훈련세트, 검증세트]
# [훈련세트, 검증세트, 훈련세트]
# [검증세트, 훈련세트, 훈련세트] -> 패리티 방식 (3-폴드 교차 검증)
# 5-폴드, 10-폴드도 있다.
# 사이킷런에는 cross_validate()라는 교차 검증 함수를 사용한다.

from sklearn.model_selection import cross_validate
# 평가 할 모델 객체를 첫번째 매개변수로 전달한다.(직접 검증세트를 떼지 않고 훈련세트 전체를 전달함)
scores = cross_validate(dt, train_input, train_target)
# scors cross_validate(dt, train_input, train_target, cv=5)
# cv 매개변수를 이용해 폴드 수를 변경한다. cv=5(기본값)
print(scores)
# fit_time(훈련시간), score_time(검증시간), test_score(최종점수) 키를 가진 딕셔너리(k : v)를 반환한다.

{'fit_time': array([0.0139308 , 0.01217937, 0.01246572, 0.01452184, 0.01275229]), 'score_time': array([0.00219154, 0.00218391, 0.00210381, 0.00211716, 0.00217152]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [ ]:
import numpy as np

print(np.mean(scores['test_score']))    # test_score 이름이지만 검증폴드의 점수이다.
# 5개의 검증 점수를 평균을 내어 최종 점수를 얻는다.-> 85.53%

0.855300214703487


In [ ]:
# 주의 : cross_validate()는 훈련세트를 섞어서 폴드를 나누지 않았다. -> 분할기 사용(splitter)
# 앞서 우리는 tarin_test_split() 함수로 전체 데이터를 섞은 후 훈련세트를 준비했다.
# 만약 교차 검증을 할 때 훈련세트를 섞으려면 분할기(splitter)를 지정해야 한다.
# 사이킷런의 분할기는 교차검증에서 폴드를 어떻게 나눌지를 결정한다.
# cross_validate()는 기본적으로 회귀모델인 KFold 분할기를 사용한다.
# 분류 모델일 경우 타킷 클래스를 골고루 나누기 위해서 StratifiedKFold를 사용한다.
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [ ]:
splitter = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)    # 10-폴드 교차 검증 수행
scores = cross_validate(dt, train_input, train_target, cv=splitter)
print(np.mean(scores['test_score']))

0.8574181117533719


In [ ]:
# 결정트리의 매개변수 값을 바꿔가며 가장 좋은 성능이 나오는 모델을 찾아봐야 한다.
# 테스트 세트를 사용하지 않고, 교차 검증을 통해서 좋은 모델을 골라야 한다.

# 하이퍼파라미터 튜닝
# 모델 파라미터 : 머신런닝 모델이 학습하는 파라미터
# 하이퍼파라미터 : 사용자가 지정해야만 하는 파라미터
# 하이퍼파라미터 튜닝 : 라이브러리가 제공하는 기본값을 그대로 사용해 모델을 훈련한다.
# 그 다음 검증세트의 점수나 교차 검증을 통해서 매개변수를 조금씩 변경 1 ~ 2, 5 ~ 6개의 매개변수를 제공한다.
# AutoML : 사람의 개입없이 하이퍼파라미터 튜닝을 자동으로 수행하는 기술

# Max_depth 를 최적으로 고정하고 min_sample_split을 바꿔가며 최적의 값을 찾는다. 면 값이 함께 변경된다.
# -> 두개의 매개변수를 동시에 바꿔가며 최적의 값을 찾아야한다. -> 다중 for문을 사용해야된다 -> (복잡해진다.)
# 사이킷런에서 제공하는 그리드 서치 (Grid Search)가 제공 -> Grid SearchCV

In [ ]:
from sklearn.model_selection import GridSearchCV    # 그리드서치크로스검증용
# min_impurity_decrease 매개변수의 최적값을 찾아보자.
# min_sample_split : 샘플을 최소한 몇개 이상이어야 split (하위 노드로 분리) 할것인지이다.
# 클수록 가지치기 (과대적합 방지), 작을수록 정확하기 (과대적합)
params = { 'min_impurity_decrease' : [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}    # 0.0001 ~ 증가 5번
# 매개변수와 탐색할 값을 리스트를 딕셔너리로 만든다.

gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)

# 결정 트리 클래스의 객체를 생성하자마자 바로 전달한다.
# 일반 모델을 훈련하는 것 처럼 fit() 매서드를 호출한다.
# 이 메서드를 호출 하면 서치 객체는 결정 트리 모델 min_impurity_decrease 값을 바꿔가면서 총 5번 수행한다.
# GridSearchCV의 cv 매개변수 기본값은 5이다. 5 * 5 = 25번 수행한다.
# n_jobs에서 병렬 실행에 사용될 CPU 코어수를 지정한다. (-1로 사용시 모든 코어)

gs.fit(train_input, train_target)  # 훈련시작
# 교차 검증에서 최적의 하이퍼파라미터를 찾으면 전체 훈련 세트로 모델을 다시 만들어야 하지만,
# 사이컷의 그리드 서치는 검증 점수가 가장 높은 모델의 매개변수 조합으로,
# 전체 훈련세트에서 자동으로 다시 모델을 훈련한다.


GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
                                                   0.0004, 0.0005]})

In [ ]:
# 이렇게 최적화되어 훈련된 모델은 GridSearchCV 클래스로 만들어진
# 객체릐 best_estimator_ 속성에 저장되어 있다.
# 최적의 매개변수는 best_params_ 속성에 저장되어 있다.
print(f"그리드 서치를 통해 찾은 최적의 모델의 정확도 : {dt.score(train_input, train_target)*100:.2f}%")

그리드 서치를 통해 찾은 최적의 모델의 정확도 : 97.06%


In [ ]:
print(f"최적의 매개변수 : {gs.best_params_}")

  # 각 매개변수에서 수행한 교차 검증의 평균 점수는 cv_results_ 속성의 'mean_test_score'키에 저장되어 있다.

최적의 매개변수 : {'min_impurity_decrease': 0.0001}


In [ ]:
# 5번의 교차 검증으로 얻은 점수를 출력해보자.
print(gs.cv_results_['mean_test_score']) # 첫 번째 값이 가장 크다.

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [ ]:
# 눈으로 보는 것보다 넘파이를 이용해 argmax() 함수를 사용하면 가장 큰 값의 인덱스를 추출할 수 있다.
# 그 다음 이 인덱스를 사용해서 params 키에 저장된 매개변수를 출력해보자.
# 이 값이 최상의 검증 점수를 만든 매개변수의 조합이 된다.

best_index = np.argmax(gs.cv_results_['mean_test_score'])    # 0.86819297
print(gs.cv_results_['params'][best_index])

# 과정 요약
# 1. 탐색할 매개변수 지정
# 2. 훈련 세트에서 그리드 서치를 수행하여 최상의 평균 검증 점수가 나오는 매개변수 조합을 찾는다.
# -> 그리드 서치에 저장한다.
# 3. 그리드 서치는 최상의 매개변수에서 전체 세트를 사용해 최종 모델을 훈련한다.
# -> 그리드 서치에 저장한다.

{'min_impurity_decrease': 0.0001}


In [ ]:
# 조금 더 복잡한 매개변수 조합 해보자.
# 노드를 분할하기 위한 불순도 감소 최소량 지정 max_depth(트리의 깊이)
# min_samples_split 노드를 나누기 위한 최소 샘플 수
# 넘파이 arange() 함수는 첫번쨰 매개변수 값에서 시작해서 두번째 매개변수를 게속 더한 배열을 만든다.

# 0.0001 ~ 0.0001까지 0.00001을 더한 배열 (두버째 매개변수는 포함되지 않는다) = 9
# 파이썬 ragne() 함수는 정수만 사용가능 하다. 5 ~ 20 까지 1씩 증가 = 15개
# 교차 검증 횟수 9 * 15 * 10 = 1350 개 * 5-폴드교차 6750개

params = {'min_impurity_decrease' : np.arange(0.0001, 0.001, 0.0001),   # 9번
          'max_depth' : range(5,20,1),
          'min_samples_split' : range(2, 100, 10)}

gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target) # 훈련시작

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [ ]:
# criterion : 분할 품질을 측정하는 기능 (default : gini)
# splitter : 각 노드에서 분할을 선택 하는 데 사용되는 전략 (default : best)
# max_depth : 트리의 최대 깊이 (값이 클수록 모델의 복잡도가 올라간다.)
# min_samples_split : 자식 노드를 분할하는데 필요한 최소 샘플 수 (default : 2)
# min_samples_leaf : 리프 노드에 있어야 할 최소 샘플 수 (default : 1)

In [ ]:
print(gs.best_params_)   # 최상의 매개변수 조합확인

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [ ]:
print(np.max(gs.cv_results_['mean_test_score']))    # 최상의 교차 점수 확인
# 개선점 : 탐색할 매개변수 간격을 0.0001 혹은 1로 설정 했는데 근거 부족 (좁히거나 넓힐 필요성이 있다.)

0.8683865773302731


In [ ]:
# 매개변수의 값이 수치 일 때 값의 범위나 간격을 미리 정하기 어려울 수 있다.
# 너무 많은 매개 변수 조건이 있어 그리드 서치 수행시간이 오래 걸린다.
# 해결 : 랜덤 서치를 사용해 보자.

# 랜덤 서치 : 매개변수의 값의 목록을 전달하는 것이 아니라 매개변수를 샘플링 할 수 있는 확률 분포도 객체를 전달한다.
# 싸이파이 : 파이썬의 핵심 과학 라이브러리로 적분, 보간, 선형대수, 확률등 포함한 수치 계산용 전용 라이브러리 (코렙에 내장)

from scipy.stats import uniform, randint
#  uniform, randint 클래스는 모두 주어진 범위에서 고르게 값을 뽑는다. (균등 분포에서 샘플링)
# randint()에서는 정수값을 추출, uniform()은 실수값을 뽑는다.

In [ ]:
rgen = randint(0, 10)    # 10개 숫자를 샘플링(학생마다 다름)
rgen.rvs(10)    # array([5, 0, 1, 0, 2, 5, 6, 5, 3, 3])
# 10개밖에 되지 않기 때문에 고르게 샘플링 되는 것 같지 않지만 샘플링 숫자를 늘리면 쉽게 확인

array([9, 2, 1, 7, 4, 9, 4, 1, 0, 2])

In [ ]:
np.unique(rgen.rvs(1000), return_counts=True) # 1000개를 샘플링해서 각 숫자의 개수를 세어봄

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([ 99, 100,  83, 106, 123, 101, 104,  98,  94,  92]))

In [ ]:
ugen = uniform(0, 1) # 실수도 테스트
ugen.rvs(10)         # 난수발생기 (랜덤과 유사하다.)

array([0.51526748, 0.29071684, 0.19520372, 0.56615693, 0.55940644,
       0.845988  , 0.08906322, 0.05668139, 0.21889946, 0.44076599])

In [ ]:
# 탐색할 매개변수의 딕서너리를 만들어 봄
# min_samples_leaf을 탐색 대상에 추가
params = {'min_impurity_decrease': uniform(0.0001, 0.001), # 0.00001 ~ 0.001의 실수값
          'max_depth': randint(20, 50),                   # 20~50 사이의 정수
          'min_samples_split': randint(2, 25),            # 2~25 사이의 정수
          'min_samples_leaf': randint(1, 25),             # 1~25 사이의 정수
          # 리프 노드가 되기 위한 최소 샘플 개수(지식 노드의 샘플 수가 이 값보다 작으면 분할 하지 않음)
          }

In [ ]:
# 샘플링 회수는 사이킷런의 램덤 서치 클래스인 RandomizedSearchCV의 n_iter 매개변수에 지정
from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params,
                        n_iter=100, n_jobs=-1, random_state=42)
# n_iter=100 총 100번을 샘플링하여 교차 검증을 수행(최적의 매개변수 조합을 찾음)

gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7b3fa839ba00>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x7b3fa839a770>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7b3fa8399990>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x7b3fa8168b50>},
                   random_state=42)

In [ ]:
print(gs.best_params_)    # 결과 최적의 매개변수 조합을 찾는다.

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [ ]:
print(np.max(gs.cv_results_['mean_test_score']) )    # 최고의 교차 검증 점수 확인

0.8695428296438884


In [ ]:
dt = gs.best_estimator_
# 최종 모델로 결정하고 테스트 세트의 성능을 확인한다.
print(dt.score(test_input, test_target))

0.86


In [ ]:
# 테스트 세트 점수는 검증세트 점수보다 조금 작은 것이 일반적이다.
# 충분히 다양한 매개변수를 테스트해서 얻은 점수니 알고 진행한다.